In [ ]:
# import necessary libraries

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv("rainfall.csv")
df

In [ ]:
df.columns.to_series().groupby(df.dtypes).groups

**Comment:**
- 'MinTemp', 'MaxTemp', 'Rainfall', 'Evaporation', 'Sunshine', 'WindGustSpeed', 'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am', 'Humidity3pm', 'Pressure9am', 'Pressure3pm', 'Cloud9am', 'Cloud3pm', 'Temp9am', 'Temp3pm' are Float in nature.
- 'Date', 'Location', 'WindGustDir', 'WindDir9am', 'WindDir3pm', 'RainToday', 'RainTomorrow' are Object in nature.
- 'Rainfall' and 'RainTomorrow' are Target Variable. 
- This dataset contain 8425 Rows and 23 Columns.


# Statistical Analysis

**Since dataset is large, Let check for any entry which is repeated or duplicated in dataset.**

In [ ]:
df.duplicated().sum()

**If we just check CSV File we can find that there are some missing value in dataset which shown fill with '?'**

Let's check how many question mark ("?") inside dataset

In [ ]:
df.isin([' ?']).sum()

In [ ]:
df.isin([' ','NA','-']).sum().any()

- We have No whitespace, NA, '-', exist in dataset.

**Lets drop duplicated entry from dataset before checking null values.**

In [ ]:
df.drop_duplicates(keep='last', inplace=True)

In [ ]:
df.shape

# Missing value check

In [ ]:
plt.figure(figsize=(10,6))
sns.heatmap(df.isnull())

In [ ]:
df.isnull().sum()

- All columns have missing values except Date and Location

In [ ]:
#Finding what percentage of data is missing from the dataset
missing_values = df.isnull().sum().sort_values(ascending = False)
percentage_missing_values =(missing_values/len(df))*100
print(pd.concat([missing_values, percentage_missing_values], axis =1, keys =['Missing Values', '% Missing data']))

**As missing values present are less than 1%. So we can directly drop these missing values.**

In [ ]:
# Converting datatype of date column
df['Date'] = pd.to_datetime(df.Date)

In [ ]:
df.info()

**Observation:**
- There 6762 rows in this dataset.
- 16 columns are Numerical variable and having float64 datatypes.
- 6 columns are categorical feature with object datatypes.

In [ ]:
# Separating numerical and categorical variable
Numerical = ['MinTemp','MaxTemp','Rainfall','Evaporation','Sunshine','WindGustSpeed','WindSpeed9am','WindSpeed3pm','Humidity9am','Humidity3pm','Pressure9am','Pressure3pm','Cloud9am','Cloud3pm','Temp9am','Temp3pm']
Categorical = ['Location','WindGustDir','WindDir9am','WindDir3pm','RainToday','RainTomorrow']

In [ ]:
# Imputating Missing value with mode for categorical features
df['WindDir9am'].fillna(df['WindDir9am'].mode()[0],inplace=True)
df['WindGustDir'].fillna(df['WindGustDir'].mode()[0],inplace=True)
df['WindDir3pm'].fillna(df['WindDir3pm'].mode()[0],inplace=True)
df['RainToday'].fillna(df['RainToday'].mode()[0],inplace=True)
df['RainTomorrow'].fillna(df['RainTomorrow'].mode()[0],inplace=True)

# Missing Value check After Imputation

In [ ]:
#Finding what percentage of data is missing from the dataset
missing_values = df.isnull().sum().sort_values(ascending = False)
percentage_missing_values =(missing_values/len(df))*100
print(pd.concat([missing_values, percentage_missing_values], axis =1, keys =['Missing Values', '% Missing data']))

In [ ]:
for col in Numerical:
    if df[col].isnull().sum() > 0:
        val = df[col].mean()
        df[col] = df[col].fillna(val)

In [ ]:
#Finding what percentage of data is missing from the dataset
missing_values = df.isnull().sum().sort_values(ascending = False)
percentage_missing_values =(missing_values/len(df))*100
print(pd.concat([missing_values, percentage_missing_values], axis =1, keys =['Missing Values', '% Missing data']))

**Comment:**

**Finally, No Missing Value is Present.**

We are Now Yes To Go Further!!!

# Statistical Matrix

In [ ]:
df.info()

In [ ]:
df.describe().T

In [ ]:
print("\033[1m"+'Minimum Rainfall :'+"\033[0m",df.Rainfall.min())
print("\033[1m"+'Maximum rainfall :'+"\033[0m",df.Rainfall.max())
print("\033[1m"+'Average Rainfall :'+"\033[0m",df.Rainfall.mean())

In [ ]:
plt.figure(figsize=(12,7))
plt.title('Distribution Rainfall')
sns.distplot(df["Rainfall"], color='b')

- Most of cases Rainfall varies between 0 to 25

In [ ]:
df['RainTomorrow'].value_counts()

In [ ]:
df.groupby('RainTomorrow')['Rainfall'].mean()

In [ ]:
print("\033[1m"+'Percentage difference of Rain :'+"\033[0m",((7.050487-1.486704)/1.486704)*100,"%")

In [ ]:
plt.figure(figsize=(8,6))
sns.boxplot(y="RainTomorrow", x="Rainfall", data=df, palette = 'hsv')

In [ ]:
labels = 'Yes','No',
fig, ax = plt.subplots()
ax.pie(df.groupby('RainTomorrow')['Pressure9am'].mean(),labels = labels,radius =2,autopct = '%2.2f%%',explode=[0.3,0.2], shadow=True,)
plt.show()

In [ ]:
df.groupby('RainTomorrow')['Pressure9am'].mean()

In [ ]:
pd.crosstab([df.RainTomorrow,df.Pressure9am],df.Rainfall, margins= True).style.background_gradient(cmap='summer_r')

In [ ]:
for i in Categorical:
    print(i)
    print(df[i].value_counts())
    print('='*100)

In [ ]:
plt.rcParams["figure.autolayout"]=True
sns.set_palette("husl")
f,ax=plt.subplots(1,2,figsize=(16,10))
df['Rainfall'].value_counts().plot.pie(autopct='%3.1f%%', textprops={'fontweight':'bold','fontsize':18}, ax=ax[0], shadow=True)
ax[0].set_title('Population Distribution', fontsize=22, fontweight='bold')
ax[0].set_ylabel('')
sns.countplot(x='RainTomorrow', data=df, ax=ax[1])
ax[1].set_title('Rainfall Distribution', fontsize=22,fontweight='bold')
ax[1].set_xlabel('Rain',fontsize=18, fontweight='bold')
plt.xticks(fontsize=18,fontweight='bold')
plt.show()

**Observation :**
- 64.1% population have 0.0 rainfall
- Our task is to predict rainTomorrow and we see that target variable rain is imbalanced.

**Exploration of RainTomorrow**

In [ ]:
print('Minimum :', df['Rainfall'].min())
print('Maximum :', df['Rainfall'].max())
print('Average :', df['Rainfall'].mean())

In [ ]:
plt.rcParams["figure.autolayout"] = True
sns.set_palette('rainbow')
plt.figure(figsize=(10,10))
df['WindGustDir'].value_counts().plot.pie(autopct='%2.1f%%', textprops ={ 'fontsize':13}, shadow=True)
plt.title('Population distribution ', fontsize=20,fontweight ='bold')
plt.tight_layout()
plt.show()

In [ ]:
pd.crosstab(df['WindGustDir'],df["Rainfall"], margins=True).style.background_gradient(cmap='summer_r')

In [ ]:
plt.rcParams["figure.autolayout"] = True
sns.set_palette('rainbow')
plt.figure(figsize=(10,10))
df['WindDir3pm'].value_counts().plot.pie(autopct='%2.1f%%', textprops ={ 'fontsize':13}, shadow=True)
plt.title('Population distribution ', fontsize=20,fontweight ='bold')
plt.tight_layout()
plt.show()

In [ ]:
pd.crosstab(df['WindDir3pm'],df["Rainfall"], margins=True).style.background_gradient(cmap='summer_r')

In [ ]:
plt.rcParams["figure.autolayout"] = True
sns.set_palette('rainbow')
plt.figure(figsize=(10,10))
df['Cloud3pm'].value_counts().plot.pie(autopct='%2.1f%%', textprops ={ 'fontsize':13}, shadow=True)
plt.title('Population distribution ', fontsize=20,fontweight ='bold')
plt.tight_layout()
plt.show()

In [ ]:
pd.crosstab(df['Cloud3pm'],df["Rainfall"], margins=True).style.background_gradient(cmap='summer_r')

In [ ]:
plt.rcParams["figure.autolayout"] = True
sns.set_palette('rainbow')
plt.figure(figsize=(10,10))
df['RainTomorrow'].value_counts().plot.pie(autopct='%2.1f%%', textprops ={ 'fontsize':13}, shadow=True)
plt.title('Population distribution ', fontsize=20,fontweight ='bold')
plt.tight_layout()
plt.show()

In [ ]:
pd.crosstab(df['RainTomorrow'],df["Rainfall"], margins=True).style.background_gradient(cmap='summer_r')

In [ ]:
# Checking the pairwise relation in the dataset.
sns.pairplot(df, hue="Rainfall",palette="husl")

In [ ]:
df.head()

# Encoding categorical data

In [ ]:
# Using Label Encoder on categorical variable
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for i in Categorical:
    df[i] = le.fit_transform(df[i])
df.head()

# Feature selection and Engineering

# 1. Outliers Detection and Removal

In [ ]:
plt.figure(figsize=(18,10),facecolor='white')
plotnumber=1

for column in Numerical:
    if plotnumber<=16:
        ax=plt.subplot(6,3,plotnumber)
        sns.boxplot(x=df[column],color='g')
        plt.xlabel(column,fontsize=20)
    plotnumber+=1
plt.show()

**From Boxplot we can see outliers exist dataset**

In [ ]:
# Dropping unnecessary columns
df.drop(['Sunshine','Evaporation','WindGustSpeed'], axis=1, inplace=True)

In [ ]:
df.drop(['Date'], axis=1, inplace=True)

**Outliers removal using Zscore method**

In [ ]:
from scipy.stats import zscore
z = np.abs(zscore(df))
threshold = 3
df1 = df[(z<3).all(axis = 1)]

print ("Shape of the dataframe before removing outliers: ", df.shape)
print ("Shape of the dataframe after removing outliers: ", df1.shape)
print ("Percentage of data loss post outlier removal: ", (df.shape[0]-df1.shape[0])/df.shape[0]*100)

df=df1.copy() # reassigning the changed dataframe name to our original dataframe name

# 2. Skewness of features

In [ ]:
plt.figure(figsize=(22,10),facecolor='white')
plotnum=1
for col in df:
    if plotnum<=19:
        plt.subplot(5,4,plotnum)
        sns.distplot(df[col],color='r')
        plt.xlabel(col,fontsize=20)
    plotnum+=1
plt.show()

In [ ]:
df.skew()

**Observation:**
- Location, RainToday,RainTomorrow,WindDir9am,WindDir3pm are skewed but as they are categorical concept of skewness doesnot mean anything to it.
- Rainfall are numerical variable with lot of zero and high number. So skewness exist in them.

# 3.Corrleation

In [ ]:
df.corr()

In [ ]:
plt.figure(figsize=(25,15))
sns.heatmap(df.corr(), vmin=-1, vmax=1, annot=True, square=True, fmt='0.3f', 
            annot_kws={'size':10}, cmap="gist_stern")
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.show()

In [ ]:
plt.figure(figsize = (18,6))
df.corr()['Rainfall'].drop(['Rainfall']).sort_values(ascending=False).plot(kind='bar',color = 'purple')
plt.xlabel(df,fontsize=15)
plt.ylabel('Income',fontsize=15)
plt.title('Correlation of features with Target Variable Income',fontsize = 18)
plt.show()

**Observation:**
- Temp9am, Location, WindSpeed9am, WindSpeed3pm are correlated with Target variable with less than 10% correlation. After checking Multicollinearity we will decide to drop these poorly correlated features or go for PCA.
- RainToday is highly correlated with Target variable.

# 4. Checking Multicollinearity between features using variance_inflation_factor

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
vif= pd.DataFrame()
vif['VIF']= [variance_inflation_factor(df.values,i) for i in range(df.shape[1])]
vif['Features']= df.columns
vif

**Strategy to Address Multicollinearity:**
- Removing Some of highly correlated features. But this will not work here as most of input features are correlated with each other moderated or poorly.
- Another way to address Multicollinearity is to scaled data and then apply PCA.

# 5.Balancing Imbalanced target feature

In [ ]:
df.RainTomorrow.value_counts()

**As Target variable data is imbalanced in nature we will need to balance target variable.**

**Balancing using SMOTE**

In [ ]:
from imblearn.over_sampling import SMOTE

In [ ]:
# Splitting data in target and dependent feature
X = df.drop(['RainTomorrow'], axis =1)
Y = df['RainTomorrow']

In [ ]:

# Oversampleing using SMOTE Techniques
oversample = SMOTE()
X, Y = oversample.fit_resample(X, Y)

In [ ]:
Y.value_counts()

**We have successfully resolved the class imbalanced problem and now all the categories have same data ensuring that the ML model does not get biased towards one category.**


# Standard Scaling

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler= StandardScaler()
X_scale = scaler.fit_transform(X)

In [ ]:
from sklearn.decomposition import PCA
pca = PCA()
#plot the graph to find the principal components
x_pca = pca.fit_transform(X_scale)
plt.figure(figsize=(10,10))
plt.plot(np.cumsum(pca.explained_variance_ratio_), 'ro-')
plt.xlabel('Number of Components')
plt.ylabel('Variance %')
plt.title('Explained variance Ratio')
plt.grid()

**Comment:**

**AS per the graph, we can see that 9 principal components attribute for 90% variation in the data. We shall pick the first 9 components for our prediction.**

In [ ]:
pca_new = PCA(n_components=9)
x_new = pca_new.fit_transform(X_scale)

In [ ]:
principle_x=pd.DataFrame(x_new,columns=np.arange(9))

# Machine learning model building

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import accuracy_score, confusion_matrix,classification_report,f1_score

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(principle_x, Y, random_state=99, test_size=.3)
print('Training feature matrix size:',X_train.shape)
print('Training target vector size:',Y_train.shape)
print('Test feature matrix size:',X_test.shape)
print('Test target vector size:',Y_test.shape)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix,classification_report,f1_score
maxAccu=0
maxRS=0
for i in range(1,200):
    X_train,X_test,Y_train,Y_test = train_test_split(principle_x,Y,test_size = 0.3, random_state=i)
    log_reg=LogisticRegression()
    log_reg.fit(X_train,Y_train)
    y_pred=log_reg.predict(X_test)
    acc=accuracy_score(Y_test,y_pred)
    if acc>maxAccu:
        maxAccu=acc
        maxRS=i
print('Best accuracy is', maxAccu ,'on Random_state', maxRS)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(principle_x, Y, random_state=44, test_size=.3)
log_reg=LogisticRegression()
log_reg.fit(X_train,Y_train)
y_pred=log_reg.predict(X_test)
print('\033[1m'+'Logistics Regression Evaluation'+'\033[0m')
print('\033[1m'+'Accuracy Score of Logistics Regression :'+'\033[0m', accuracy_score(Y_test, y_pred))
print('\033[1m'+'Confusion matrix of Logistics Regression :'+'\033[0m \n',confusion_matrix(Y_test, y_pred))
print('\033[1m'+'classification Report of Logistics Regression'+'\033[0m \n',classification_report(Y_test, y_pred))

In [ ]:
# Finding Optimal value of n_neighbors for KNN

from sklearn import neighbors
from math import sqrt
from sklearn.metrics import mean_squared_error
rmse_val = [] #to store rmse values for different k
for K in range(15):
    K = K+1
    model = neighbors.KNeighborsClassifier(n_neighbors = K)

    model.fit(X_train,Y_train)  #fit the model
    y_pred=model.predict(X_test) #make prediction on test set
    error = sqrt(mean_squared_error(Y_test,y_pred)) #calculate rmse
    rmse_val.append(error) #store rmse values
    print('RMSE value for k= ' , K , 'is:', error)

In [ ]:
#plotting the rmse values against k values -
plt.figure(figsize = (8,6))
plt.plot(range(15), rmse_val, color='blue', linestyle='dashed', marker='o', markerfacecolor='green', markersize=10)

**Comment -**
- At k=1, we get the minimum RMSE value which approximately 0.36647465376087945, and shoots up on further increasing the k value. We can safely that k=1 will give us the best result in this case.

**Applying other classification algorithm**

In [ ]:
model=[ LogisticRegression(),
        SVC(),
        DecisionTreeClassifier(),
        KNeighborsClassifier(n_neighbors = 1),
        RandomForestClassifier(),
        ExtraTreesClassifier()]
        
for m in model:
    m.fit(X_train,Y_train)
    y_pred=m.predict(X_test)
    print('\033[1m'+'Classification ML Algorithm Evaluation Matrix',m,'is' +'\033[0m')
    print('\033[1m'+'Accuracy Score :'+'\033[0m\n', accuracy_score(Y_test, y_pred))
    print('\033[1m'+'Confusion matrix :'+'\033[0m \n',confusion_matrix(Y_test, y_pred))
    print('\033[1m'+'Classification Report :'+'\033[0m \n',classification_report(Y_test, y_pred))
    print('\n')
    print('=================================================================')

# CrossValidation:

In [ ]:
from sklearn.model_selection import cross_val_score
model=[LogisticRegression(),
        SVC(),
        DecisionTreeClassifier(),
        KNeighborsClassifier(n_neighbors = 1),
        RandomForestClassifier(),
        ExtraTreesClassifier()]

for m in model:
    score = cross_val_score(m, principle_x, Y, cv =5)
    print('\n')
    print('\033[1m'+'Cross Validation Score', m, ':'+'\033[0m\n')
    print("Score :" ,score)
    print("Mean Score :",score.mean())
    print("Std deviation :",score.std())
    print('\n')
    print('============================================================')

# Hyper Parameter Tuning : GridSearchCV

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
parameter= {'criterion' : ['gini', 'entropy'],
              'min_samples_split':[3,5,8],
              'max_depth' : [20,30,40],
              'n_estimators' : [100, 150, 200]
             }

In [ ]:
GCV = GridSearchCV(ExtraTreesClassifier(),parameter,verbose=2)
GCV.fit(X_train,Y_train)

In [ ]:
GCV.best_params_

# Final Model

In [ ]:
Final_mod = ExtraTreesClassifier(criterion='entropy',n_estimators= 150, max_depth=40 ,min_samples_split= 3)
Final_mod.fit(X_train,Y_train)
y_pred=Final_mod.predict(X_test)
print('\033[1m'+'Accuracy Score :'+'\033[0m\n', accuracy_score(Y_test, y_pred))

# Saving model

In [ ]:
import joblib
joblib.dump(Final_mod,'Rainfall_whether_forecast.pkl')